In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math
import planet_sampler_pop as ps
import pandas as pd
from scipy import stats  
import scipy.optimize as opt
%matplotlib inline

In [2]:
weiss_dat = pd.read_csv("weiss_tables.csv")

In [20]:
def snr_weiss(R_p, R_star, M_star, period, CDPP_6h):
    """
    snr_weiss : Computes the expected signal to noise ratio of a planet with size R_p and orbital period
    period orbiting a star with bulk density rho_star and radius R_star and a 6hr combined differential photometric precision 
    CDPP_6hr. This definition is taken from Weiss et al. 2018 https://iopscience.iop.org/article/10.3847/1538-3881/aa9ff6
    #####
    Parameters : 
    R_p : planet radius in Earth radius
    R_star : host star radius in solar radius
    M_star: host star mass in stellar mass
    period : period in days
    CDPP_6h : combined differential photometric precision over 6hr
    
    returns the value of SN ratio
    """
    R_sun = 695700
    R_earth = 6378
    M_sun = 1.989 * 10**30
    R_p_km = R_p * R_earth
    R_star_km = R_sun * R_star
    M_star_kg = M_star * M_sun
    rho_star = M_star/(R_star**3)
    depth = 1000000*(R_p_km / R_star_km)**2
    T = 13 * (period/365.24)**(1/3) * (rho_star)**(-1/3) # Parameter T in hours
    #print("period : " + str(period))
    #print("radius : " + str(R_p))
    #print("duration : " + str(T))
    #print("depth : " + str(depth))
    return  ((depth * math.sqrt(3.5 *365.24/ (period ))) / (CDPP_6h * math.sqrt(6/T)))


def swapping_criterion(R_p, R_star, M_star, period, CDPP_6h, snr_threshold = 10):
    return (snr_weiss(R_p, R_star, M_star, period, CDPP_6h) >= snr_threshold)
    
def gaussian_fit(x, mu, sigma):
    return 1/math.sqrt(2*math.pi*sigma**2) * np.exp(-(x - mu)**2/(2*sigma**2))
def skew_gaus(x, mean, skew, sigma):
    return stats.skewnorm.pdf(x, skew, mean, sigma)

In [22]:
count = 0
tot_count = 0
two = 0
three = 0
for star in weiss_dat['host_star'].unique():
    temp_syst = weiss_dat[weiss_dat['host_star'] == star]
    temp_syst = temp_syst.iloc[temp_syst['planet_period'].argsort()]
    
    for i in range(len(temp_syst) -1 ):
        tot_count += 1
        #print(temp_syst['KOIS'])
        if(swapping_criterion(min(temp_syst['planet_radius'].iloc[i], temp_syst['planet_radius'].iloc[i+1]), temp_syst['star_radius'].iloc[i], temp_syst['star_mass'].iloc[i], temp_syst['planet_period'].iloc[i+1], temp_syst['CDPP_6h'].iloc[i], 10)):
            count +=1
            if(len(temp_syst) >=2):
                two +=1
            if(len(temp_syst) >= 3):
                three +=1 
        else:
            qqq = snr_weiss(min(temp_syst['planet_radius'].iloc[i], temp_syst['planet_radius'].iloc[i+1]), temp_syst['star_radius'].iloc[i], temp_syst['star_mass'].iloc[i], temp_syst['planet_period'].iloc[i+1], temp_syst['CDPP_6h'].iloc[i])
print(count, tot_count)
print(two, three)

491 554
491 299


In [176]:
554-491

63

In [101]:
weiss_dat['KOIP'][weiss_dat['KOIP'][-6:] == '205.02']

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [141]:
weiss_dat[weiss_dat['KOIS'] == 'K00488']

,Unnamed: 0,KOIS,host_star,KOIP,Kepname,star_mass,star_radius,CDPP_6h,b,planet_period,planet_radius,e_Rp,T,RpAvg,Prat,Delta
